The Code below Scraps the Voa for available links and scaps the news thereafter

In [ ]:
from urllib import request
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}
soupx = []

In [ ]:
categories = ["Najeriya", "Afirka", "Amurka", "Sauran Duniya", "Kiwon Lafiya"] #get all the news categories
size = [101,101,101,101,54]  #get the number of pages each link has
home = "https://www.voahausa.com" #the link to the home page
#get the link to each category append /z/2866?p= at the end
links = ['https://www.voahausa.com/z/2866?p=','https://www.voahausa.com/z/2863?p=','https://www.voahausa.com/z/5225?p=','https://www.voahausa.com/z/2870?p=','https://www.voahausa.com/z/2864?p=']

In [ ]:
#get all available links from the voa website
def getAllLinks(category, sizes, link):
  links = []
  for cat in range(len(category)):
    print("Getting all links from ", category[cat] , " new category")
    siz = sizes[cat]
    ls = []
    for ln in range(siz):
      href = link[cat]+str(ln)
      #print(href)
      page_request = request.Request(href, headers=headers)
      page = request.urlopen(page_request)
      soup = BeautifulSoup(page, 'html.parser')
      result_set = soup.findAll("div", {"class":"media-block__content media-block__content--h media-block__content--h-xs"})
      ls.extend([x.a['href'] for x in result_set ])
    links.extend(ls)
    print(category[cat], "returned ", len(ls), " links ...")
  print("Got ", len(links), " links in all .. and  ", len(set(links)), "unique links ....")
  return list(set(links))



In [ ]:
allinks = getAllLinks(categories, size, links)

Getting all links from  Najeriya  new category
Najeriya returned  1111  links ...
Getting all links from  Afirka  new category
Afirka returned  1111  links ...
Getting all links from  Amurka  new category
Amurka returned  1111  links ...
Getting all links from  Sauran Duniya  new category
Sauran Duniya returned  1111  links ...
Getting all links from  Kiwon Lafiya  new category
Kiwon Lafiya returned  591  links ...
Got  5035  links in all .. and   4871 unique links ....


In [ ]:
allinks

['/a/wane-ne-mike-pompeo/4298149.html',
 '/a/covid-19-mutum-sama-da-100-000-suka-mutu-a-amurka/5438912.html',
 '/a/yan-boko-haram-2-daga-chadi-sun-mika-wuya-ga-dakarun-najeriya/5312950.html',
 '/a/mummunar-girgizar-kasa-ta-abka-ma-kasar-albania/5181932.html',
 '/a/cutar-ebola-ta-kashe-akalla-mutane-2000-a-yammacin-afierka/2441861.html',
 '/a/an-saki-sambo-dasuki/5218874.html',
 '/a/turkiya-zata-kara-haraji-kan-kayan-dake-shiga-kasarta-daga-amurka/4449876.html',
 '/a/an-gina-kauyen-mata-zalla-a-syria/4664420.html',
 '/a/ana-shirin-fara-taron-yaki-da-kwalara-na-voa-a-bauchi-103606019/1372043.html',
 '/a/jamian-koriya-ta-arewa-da-ta-kudu-sun-gana-domin-karfafa-zumunci/4508655.html',
 '/a/fadar-vatican-ta-tsige-wani-limamin-katolika-a-amurka/4790570.html',
 '/a/yan-sanda-sunyi-awon-gaba-da-shaharraren-mawaki-bobi-wine/4580083.html',
 '/a/an-nada-mai-martaba-aminu-ado-bayero-a-matsayin-sarkin-kano/5321372.html',
 '/a/amurka-ta-kashe-mayakan-al-shabab-takwa-a-jiya-asabar/4702684.html',
 '/a/

In [ ]:
#scaps each new article for the title, date, introduction and content
def page_content(soups):
  if soups.find("h1", {"class":"title pg-title"})!= None:
    title = soups.find("h1", {"class":"title pg-title"}).text.replace("\t","").replace("\n","")
  elif soups.find("h1", {"class":"title pg-title pg-title--featured"})!= None:
    title = soups.find("h1", {"class":"title pg-title pg-title--featured"}).text.replace("\t","").replace("\n","")
  else:
    title = "" 
  if soups.find("span", {"class":"date"})!= None:
    time = ""#soups.find("span", {"class":"date"}).text
  else:
    time  = "" 
 
  introstring=""
  if soups.find("div", {"class":"intro intro--bold"})!=None:
    intro_res = soups.find("div", {"class":"intro intro--bold"}).findAll('p')
    #for x in intro_res:
    #  introstring+=x.text.replace(u'\xa0', u' ').replace('\n'," ")+" \n"
    introstring = " ".join([ x.text.replace(u'\xa0', u' ').replace('\n'," ")+" \n" for x in intro_res])


  if soups.find("div", {"class":"wsw"})!=None:
    result = soups.find("div", {"class":"wsw"}).findAll('p')
    #txtstring=""
    #for x in result:
    #print (x.text)
     # txtstring+=x.text.replace(u'\xa0', u' ').replace('\n'," ")+" \n"
    texts = " ".join([ x.text.replace(u'\xa0', u' ').replace('\n'," ")+" \n" for x in result])
    texts = texts.strip()
  else:
    texts = "" 
  finaltext = introstring+"\n"+texts
  return title, time, finaltext.strip()

In [ ]:
def scrap_pages(alllinks):
  titles, times, texts = [],[],[]
  count = 0
  for urls in alllinks:
    page_request = request.Request(home+urls, headers=headers)
    page = request.urlopen(page_request)
    soup = BeautifulSoup(page, 'html.parser')
    title, time, text = page_content(soup)
    titles.append(title)
    times.append(time)
    texts.append(text)
    count = count + 1
    progresse = (count/len(alllinks) * 100) 
    if progresse  % 10 == 0:
      print ("Scrapped ", progresse, "% of the articles")
  return titles, times, texts
 

In [ ]:
titles, times, texts = scrap_pages(allinks)

Scrapped  10.0 % of the articles
Scrapped  20.0 % of the articles
Scrapped  30.0 % of the articles
Scrapped  40.0 % of the articles
Scrapped  50.0 % of the articles
Scrapped  60.0 % of the articles
Scrapped  70.0 % of the articles
Scrapped  80.0 % of the articles
Scrapped  90.0 % of the articles
Scrapped  100.0 % of the articles


In [ ]:
# create a dictionary of the top
d = {'Title':titles,'Date':times,"Text":texts}

In [ ]:
# Load pandas as pd
import pandas as pd
df = pd.DataFrame(d)
df.to_csv (r'voa_hausa.csv', index = False, header=True)

In [ ]:
df

,Title,Date,Text
0,Wane Ne Mike Pompeo?,,"Michael Richard Pompeo, wanda aka fi sani da M..."
1,"COVID-19: Mutum Sama Da 100,000 Suka Mutu a Am...",,Adadin wadanda suka mutu a Amurka sanadiyyar c...
2,'Yan Boko Haram 2 Daga Chadi Sun Mika Wuya Ga ...,,Wasu mayakan Boko haram ‘yan asalin kasar Chad...
3,Mummunar Girgizar Kasa Ta Abka Ma kasar Albania,,"Girgizar kasar, wacce ta kai 6.4 a ma'aunin gi..."
4,"Cutar Ebola ta Kashe Akalla Mutane 2,000 a Yam...",,Hukumomi a Saliyo sun umurci mutane su dakata ...
...,...,...,...
95,Jihar Bauchi Tayi Barazanar Korar Ma’aikatan D...,,Gwamnatin jihar Bauchi tayi barazanar korar du...
96,Tsohon Jarumin Bollywood Irfan Khan Ya Mutu,,Tsohon jarumin masana’antar fina-finai ta Boll...
97,Birtaniya Ta Nemi Jiragen Ruwanta Su Kaucewa M...,,Hukumomin tsaron ruwan Iran sun ba da umurnin ...
98,Nijar: Gwamnati Ta Kafa Dokar Ta Baci A Yankin...,,A ci gaba da daukar matakan dakile hanyoyin da...
